In [112]:
import numpy as np
import pandas as pd

from helpers import create_polling_id

In [113]:
year = 2000

In [114]:
df = pd.read_csv("data/United States/countypres_2000-2020.csv")
df["county_fips"] = df["county_fips"].fillna(0).astype(int)
df = df[df["year"] == year]
df.head()

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL


In [115]:
df["polling_id"] = create_polling_id(df, ["state_po", "county_fips"])

In [116]:
df["rank"] = df.groupby(["polling_id"])["candidatevotes"].rank("min", ascending=False).astype(int)

In [117]:
tt = df.groupby(["polling_id", "candidate"]).agg({"candidatevotes": "sum"})
tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate"]]

C:\Users\cnava\AppData\Local\Temp\ipykernel_23200\3574999491.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  tt["rate"] = tt.groupby(level=[0]).apply(lambda x: x/x.sum())


In [118]:
df = pd.merge(df, tt, on=["polling_id", "candidate"])

In [119]:
df_filtered = df[["polling_id", "candidate", "candidatevotes", "rank", "rate"]].copy()
df_filtered = df_filtered.rename(columns={"candidatevotes": "value"})
df_filtered["candidate"] = df_filtered["candidate"].apply(lambda x: x.strip())

df_filtered.to_csv(f"data_output/United States/{year}_first_round.csv.gz", compression="gzip", index=False)

In [120]:
df_location = df[["polling_id", "state", "state_po", "county_name", "county_fips"]].drop_duplicates()

df_location.to_csv(f"data_output/United States/{year}_first_round_location.csv.gz", compression="gzip", index=False)